In [1]:
from glob import glob
import pandas as pd
paths = glob('./data/*/*/*')
filedata = pd.DataFrame([path.split('/')[2:] for path in paths], columns=['subject', 'pose', 'filename'])
filedata['path'] = paths
filedata = filedata.sort_values(['subject', 'pose']).reset_index(drop=True)
filedata
data = []
for index, value in filedata.iterrows():
    df = pd.read_pickle(value['path'])
    df = df.rename_axis('time').reset_index()
    df['id'] = index
    cols = df.columns.to_list()
    df = df[cols[-1:] + cols[:-1]]
    data.append(df)
data = pd.concat(data)
data = data.reset_index(drop=True)
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [2]:
import torch

class FSRDataset(torch.utils.data.Dataset):
    def __init__(self, X_df, y_df, index):
        assert(len(X_df) == len(y_df))
        self.X_df = X_df
        self.y_df = y_df
        self.index = index

    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, idx):
        import numpy as np
        X = self.X_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        y = self.y_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        return X, y

In [3]:
def get_index_splited_by_time(data, test_size=None):
    from sklearn.model_selection import train_test_split
    train_indexes = []
    test_indexes = []
    for _, group in data.groupby('id'):
        train_index, test_index = train_test_split(group.index, test_size=0.2, shuffle=False)
        train_indexes.append(train_index)
        test_indexes.append(test_index)
    return train_indexes, test_indexes

In [4]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        self.encoder = torch.nn.LSTM(input_size, hidden_size, num_layer)
        self.decoder = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
import torch
from ray.air import session
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ray.train.torch
import ray.tune
import ray.air
import numpy as np
import model as net
import torch.utils.data
from ray.tune.schedulers import ASHAScheduler
from sklearn.preprocessing import StandardScaler

def train_loop_per_worker(config, data):
    model_name = config['model_name']
    model_args = config['model_args']
    criterion_name = config['criterion_name']
    optimizer_name = config['optimizer_name']
    lr = config['lr']

    model = getattr(net, model_name)(**model_args)
    criterion = getattr(torch.nn, criterion_name)()
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    train_index, test_index = get_index_splited_by_time(data)
    scaler = StandardScaler()
    scaler.fit_transform(data)
    train_dataset = FSRDataset(data['FSR_for_force'], data['force'], train_index)
    test_dataset = FSRDataset(data['FSR_for_force'], data['force'], test_index)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
    
    while True:
        model.train()
        criterion.train()
        for X, y in train_loader:
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        criterion.eval()
        with torch.no_grad():
            mae = []
            mse = []
            num = []
            for X, y in test_loader:
                pred = model(X)
                mae.append(mean_absolute_error(y, pred.cpu().detach()))
                mse.append(mean_squared_error(y, pred.cpu().detach()))
                num.append(len(y))
            mae = np.array(mae)
            mse = np.array(mse)
            num = np.array(num)
            mae = (mae * num).sum() / sum(num)
            mse = (mse * num).sum() / sum(num)
            
            rmse = mse ** 0.5
            checkpoint = ray.train.torch.TorchCheckpoint.from_model(model)
            session.report({'metric/mae': mae, 'metric/rmse': rmse}, checkpoint=checkpoint)

scheduler = ASHAScheduler(
    max_t=100,
    grace_period=1,
    reduction_factor=2,
    brackets=1,
    
)
tuner = ray.tune.Tuner(
    trainable=ray.tune.with_parameters(train_loop_per_worker, data=data),
    tune_config=ray.tune.TuneConfig(
        metric='metric/rmse',
        mode='min',
        scheduler=scheduler,
        num_samples=20,
    ),
    param_space={
        'lr':ray.tune.loguniform(1e-5, 1e-2),
        'model_name':'LSTM',
        'model_args':{
            'input_size':6, 
            'hidden_size':ray.tune.choice([8, 16, 32, 64, 128, 256, 512, 1024]), 
            'num_layer':ray.tune.randint(1, 8), 
            'output_size':6,
        },
        'criterion_name': 'MSELoss',
        'optimizer_name': 'Adam',
    },
    run_config=ray.air.RunConfig(
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=2,
            checkpoint_score_attribute='metric/rmse',
            checkpoint_score_order='min',
        ),
    ),
)
results = tuner.fit()

2023-06-28 13:21:47,590	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-06-28 13:21:49,472	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


Trial name,date,done,hostname,iterations_since_restore,metric/mae,metric/rmse,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_loop_per_worker_4d252_00001,2023-06-28_13-22-20,True,DESKTOP-0P789CI,2,236.426,553.314,172.26.215.93,127988,True,19.8967,10.4397,19.8967,1687926140,2,4d252_00001
train_loop_per_worker_4d252_00002,2023-06-28_13-23-12,False,DESKTOP-0P789CI,33,236.099,552.125,172.26.215.93,127989,True,71.8465,2.53248,71.8465,1687926192,33,4d252_00002
train_loop_per_worker_4d252_00004,2023-06-28_13-23-12,False,DESKTOP-0P789CI,16,226.194,487.518,172.26.215.93,127991,True,71.5343,5.10076,71.5343,1687926192,16,4d252_00004
train_loop_per_worker_4d252_00006,2023-06-28_13-22-05,True,DESKTOP-0P789CI,1,236.649,553.436,172.26.215.93,127993,True,4.15523,4.15523,4.15523,1687926125,1,4d252_00006
train_loop_per_worker_4d252_00009,2023-06-28_13-22-51,True,DESKTOP-0P789CI,1,236.467,553.433,172.26.215.93,127988,True,31.0786,31.0786,31.0786,1687926171,1,4d252_00009
